# Janssen & Rollins ABM

This project constitutes an attempt to replicate the ABM from the paper "Evolution of cooperation in asymmetric commons dilemmas" by Marco A. Janssen and Nathan D. Rollins (2011). 
See http://www.sciencedirect.com/science/article/pii/S0167268111002599 for the paper. 

This version is only preliminary and still at an early stage. The code has not yet been cleaned up completely and still contains some unnecessary variables and functions. This will be gradually improved during the next weeks.

## Basic Structure

This replication model relies on the Project Mesa Framework (see github.com/projectmesa).
To get an understanding of how an ABM is built with mesa, it is useful to start with the basic blocks: the Agent and Model classes. These are imported from the mesa module.

In [ ]:
from mesa import Model, Agent

At the most basic level, an agent is initialized with a unique agent ID, and a variable where some information can be stored; as an example, each agent might have a score that is increased as the model runs.

In [ ]:
class Resource_Agent(Agent):
    
     def __init__(self, unique_id):
        self.unique_id = unique_id
        self.score = 0

The Model class will later create a number of agents. To do this, both the number of agents and a function to create them will be needed:

In [ ]:
class Resource_Model(Model):
    
    
    def __init__(self, N):
        self.num_agents = N
        self.create_agents()
        
    def create_agents(self):
        for i in range(self.num_agents):
            a = Resource_Agent(i)

The above already is sufficient to create a working model, although the created agents will not do anything yet. A model with 5 agents can be created and run as follows:

In [ ]:
test_model_one = Resource_Model(5)
test_model_one.create_agents()

## Adding a Scheduler

In order to have the agents perform a certain action in each step of the model, a step function will have to be added to the Agent class. As an example, the agents will just increase their own score by 1 each round.

In the Model class, it will have to be defined in what order the agents are activated, and how the model behaves at each step. Mesa includes several schedules, AAAAthe simplest being the BaseScheduler - the agents are activated in the order they were created each round.

In [ ]:
from mesa import Model, Agent
from mesa.time import RandomActivation

class Resource_Agent(Agent):
    
    def __init__(self, unique_id):
        self.unique_id = unique_id
        self.score = 0
        
    def step(self, model):
        self.score += 1

class Resource_Model(Model):
    
    def __init__(self, N):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.create_agents()
        
    def create_agents(self):
        for i in range(self.num_agents):
            a = Resource_Agent(i)
            self.schedule.add(a)
            
    def step(self):
        self.schedule.step()

    def run_model(self, steps):
        for i in range(steps):
            self.step()

This model can now be run, with 5 agents increasing their score each round, which will be repeated for 10 rounds.

In [ ]:
model = Resource_Model(5)
model.create_agents()
model.run_model(10)

## Fitting the Model to Janssen & Rollins'

In order to recreate the desired model, Agent and Model class will have to be modified to fit Janssen&Rollins Public Resource Game. The model's structure with two sub-steps per round will require changes to the Scheduler class as well.
In this first approach, the algorithm by which agents respond to each other will be omitted, and a contribution and share takeout level will be chosen at random.

### Agent Class

In the game's first round, each agent is endowed with a certain number of tokens (10), some of which can be contributed to the common public resource (CPR). Depending on the amount of contributions, this resource will be multiplied by some factor. In the second round, the agents take turns taking out some share of the CPR - the first agent gets to pick first, and so forth, until the last agent. Each step, contribution and takeout share will be chosen at random. The agents store the tokens they acquired during the game in a score variable.

In [ ]:
import random
from mesa import Model, Agent

class Resource_Agent(Agent):

    def __init__(self, unique_id):
        self.unique_id = unique_id
        self.score = 0
        self.endowment = 10
        self.contribution = random.randrange(0, 10, 1)
        self.takeout_share = (random.randrange(0, 10, 1)) / 10
        
    def first_round(self, model):
        self.endowment = 10
        self.contribution = random.randrange(0, 10, 1)
        self.score += (self.endowment - self.contribution)
        
    def second_round(self, model):
        self.takeout_share = (random.randrange(0, 10, 1)) / 10
        takeout = round(model.resource * self.takeout_share)
        self.score += takeout
        model.resource -= takeout

### Model Class

The model class will determine the amount of tokens currently in the CPR, create a number of agents, and run for a number of steps. Janssen&Rollins describe an S-shaped function for the generated resource: more than one contributing agent is necessary to create a CPR. Mesa features a class for data colletion - this will be used to visualize the model's outcome in the end.

In [ ]:
from mesa.datacollection import DataCollector

class Resource_Model(Model):
    
    def __init__(self, N):
        self.num_agents = N
        self.schedule = BaseScheduler(self)
        self.create_agents()
        self.resource = 0
        #creating a variable to determine the amount of CPR generated in this round:
        self.sum_of_contributions = 0
        #data collector
        ar = {"Score": lambda ar: ar.score}
        self.dc = DataCollector(agent_reporters = ar)
        
    def create_agents(self):            
        for i in range(self.num_agents):
            a = Resource_Agent(i)
            self.schedule.add(a)
            
    def step(self):
        self.schedule.step()
        #collecting data
        self.dc.collect(self)
        #resetting the sum of contributions needed to determine the amount of CPR:
        self.sum_of_contributions = 0
        
    def run_model(self, steps):
        for i in range(steps):
            self.step()
            
    def created_resource(self, sum_of_contributions):
        if sum_of_contributions >= 0 and sum_of_contributions <= 10:
            produced_resource = 0
        elif sum_of_contributions >10 and sum_of_contributions <= 15:
            produced_resource = 5
        elif sum_of_contributions >15 and sum_of_contributions <= 20:
            produced_resource = 20
        elif sum_of_contributions >20 and sum_of_contributions <= 25:
            produced_resource = 40
        elif sum_of_contributions >25 and sum_of_contributions <= 30:
            produced_resource = 60
        elif sum_of_contributions >30 and sum_of_contributions <= 35:
            produced_resource = 75
        elif sum_of_contributions >35 and sum_of_contributions <= 40:
            produced_resource = 85
        elif sum_of_contributions >40 and sum_of_contributions <= 45:
            produced_resource = 95
        elif sum_of_contributions >45 and sum_of_contributions <= 50:
            produced_resource = 100
        return produced_resource

### Scheduler

Due to the Model's design with two rounds in each step, the scheduler class needs to be modified. Also, after each first round, the model's created_resource() function will be called to determine the amount of CPR generated.

In [ ]:
class BaseScheduler(object):
    model = None
    steps = 0
    time = 0
    agents = []

    def __init__(self, model):
        self.model = model
        self.steps = 0
        self.time = 0
        self.agents = []

    def add(self, agent):
        self.agents.append(agent)

    def remove(self, agent):
        while agent in self.agents:
            self.agents.remove(agent)

    def step(self):
    #Modified part.
        for agent in self.agents:
            agent.first_round(self.model)
        self.model.resource = self.model.created_resource(self.model.sum_of_contributions)
        for agent in self.agents:            
            agent.second_round(self.model)
        self.steps += 1
        self.time += 1

    def get_agent_count(self):
        return len(self.agents)

As before, this model will now be run for 10 steps including 5 agents:

In [ ]:
agents = 5
runs = 10

test_model = Resource_Model(agents)
test_model.run_model(runs)

Since the data collector collects data on each agents score, the scores of the five agents can be shown in a table using matplotlib.

In [ ]:
import matplotlib.pyplot as plt

data = test_model.dc.get_agent_vars_dataframe()
data.reset_index(inplace=True)
last_step_data = data[data.Step == (runs - 1)]

x = range(agents)
plt.ylabel("Score")
plt.xlabel("Agent No.")
plt.bar(x, last_step_data.Score, align = "center")
plt.show()

## Algorithm

The model as described above is running like Janssen&Rollins' model in its basic structure, but since all moves are selected from a uniform random distribution, no useful data can be generated. To recreate the probabilistic choice function by which agents react to each other in the original model, more functions need to be implemented. This is done mainly in the Agent class; the Model class essentially remains unchanged.

### Modifications to the Agent class

As a first step, the model needs to be initialized with additional variables to store the model's parameters: alpha, beta and eta. Contribution and Takeout share in the game's first step remain uniformly random for now.

In [ ]:
from mesa import Model, Agent

class Resource_Agent(Agent):

    def __init__(self, unique_id, alpha, beta, eta):
        self.unique_id = unique_id
        self.score = 0
        self.endowment = 10
        self.contribution = random.randrange(0, 10, 1)
        self.takeout_share = (random.randrange(0, 10, 1)) / 10
        self.alpha = alpha
        self.beta = beta
        self.eta = eta

In the first_round and second_round functions, a list of probabilities from which a move is chosen replaces the uniformly random move choice. This list of probabilities is the heart of the model. It will be shown later how this list is generated.

In [ ]:
    def first_round(self, model):
        self.endowment = 10
        self.contribution = np.random.choice(11, p = self.k_probability_list_function(model))
        self.score += (self.endowment - self.contribution)
        
    def second_round(self, model):
        self.takeout_share = ((np.random.choice(11, p = self.l_probability_list) / 10))
        takeout = round(model.resource * self.takeout_share)
        self.score += takeout
        model.resource -= takeout

In the model, a hypothetical utility for each possible amount of contribution to the CPR is calculated, dependent on the choices of Contribution and Share Takeout level of the other agents in the previous round. The utility of each k is "transformed" and divided by the sum of the "transformed" utilities for all ks. The transformation is done by multiplying the utility by the model parameter eta, and calculating the exponential function for this - 
See formula (4) in the Janssen&Rollins paper. 
Note that the following functions will later need to be reversed in order to make them callable for each other, but the approach is more easily understandable by introducing them in reverse order.

In [ ]:
    def k_probability_list_function(self, model):
        probability_list = []
        for i in range(0, 11, 1):
            probability_list.append(self.probability_of_k(i, model))
        return probability_list
    
    def probability_of_k(k, model):
        utility_of_k = self.utility_transformation(k, model)
        probability = utility_of_k / self.transformed_utility_sum(model)
        return probability
    
    def utility_transformation(self, k, model):
        transformed_utility = math.exp(self.eta * (self.determine_player_utility(k, model)))
        return transformed_utility
    
    def transformed_utility_sum(self, model):
        utility_sum = 0
        for i in range(0, 11, 1):
            calculation = self.utility_transformation(i, model)
            utility_sum += calculation
        return utility_sum   

For a given hypothetical utility for each k, each agent could now pick a contribution by probabilistic choice. In order for this to work, functions to determine hypothetical utilites for each k given the previous round's contributions and takeout shares of the other agents.
The utility is dependend on an agent's earnings, the parameters alpha and beta, and the average earnings of the other players. See formula (3) in the J&R paper for this.

In [ ]:
    def utility_of_k(self, k, model):
        if self.determine_wage(k, model) > self.determine_average_wage(k, model):
            utility = self.determine_wage(k, model) - (self.alpha * (self.determine_wage(k, model) - self.determine_average_wage(k, model)))
        elif self.determine_wage(k, model) == self.determine_average_wage(k, model):
            utility = self.determine_wage(k, model)
        else:
            utility = self.determine_wage(k, model) + (self.beta * (self.determine_average_wage(k, model) - self.determine_wage(k, model)))
        return utility

In order to calculate the utility, additional functions to determine the wages for each respective player, as well as the average wage are needed. As follows from formula (4), the hypothetical wage for each contribution level k is dependent on the CPR that would be generated if all agents contributed the same amount as in the previous round, and the agent's upstream agents took out the same share as in the previous round - that is, the agent's potential takeout from the CPR.

In [ ]:
    def determine_wage(self, k, model):
        potential_takeout = self.determine_available_resource(k, model) * self.upstream_players_takeout_level_combined(model)
        wage_player = (self.endowment - k) + potential_takeout
        return wage_player

As mentioned, the potentially available resource is determined by the CPR generated by all other agents - upstream and downstream - and the takeout share of the other players. The generated CPR is subject to the S-curve transformation described earlier, which is stored in the model class.

In [ ]:
    def determine_available_resource(self, k, model):
        contributions = self.last_round_sum_of_upstream_players_contributions(model) + self.last_round_downstream_players_contributions(model) + k
        resource = model.created_resource(contributions)
        available_resource = resource - self.upstream_players_takeout_sum(model)
        return available_resource

In order to make the abovewritten function work, 

### Modifications to the Model class

Inside the Model class, additional variables to pass on the parameters to the Agent class are the only modification.

In [ ]:
class Resource_Model(Model):
    
    def __init__(self, N, alpha, beta, eta):
        self.num_agents = N
        self.alpha = alpha
        self.beta = beta
        self.eta = eta
        self.schedule = BaseScheduler(self)
        self.create_agents()
        self.resource = 0
        self.sum_of_contributions = 0
        ar = {"Score": lambda ar: ar.score}
        self.dc = DataCollector(agent_reporters = ar)
        
    def create_agents(self):            
        for i in range(self.num_agents):
            a = Resource_Agent(i)
            self.schedule.add(a)
            
    def step(self):
        self.schedule.step()
        self.dc.collect(self)
        self.sum_of_contributions = 0
        
    def run_model(self, steps):
        for i in range(steps):
            self.step()
            
    def created_resource(self, sum_of_contributions):
        if sum_of_contributions >= 0 and sum_of_contributions <= 10:
            produced_resource = 0
        elif sum_of_contributions >10 and sum_of_contributions <= 15:
            produced_resource = 5
        elif sum_of_contributions >15 and sum_of_contributions <= 20:
            produced_resource = 20
        elif sum_of_contributions >20 and sum_of_contributions <= 25:
            produced_resource = 40
        elif sum_of_contributions >25 and sum_of_contributions <= 30:
            produced_resource = 60
        elif sum_of_contributions >30 and sum_of_contributions <= 35:
            produced_resource = 75
        elif sum_of_contributions >35 and sum_of_contributions <= 40:
            produced_resource = 85
        elif sum_of_contributions >40 and sum_of_contributions <= 45:
            produced_resource = 95
        elif sum_of_contributions >45 and sum_of_contributions <= 50:
            produced_resource = 100
        return produced_resource